In [3]:
from tqdm import tqdm

def generate_finger_start(n: int, m: int, i: int) -> int:
    """Генерация стартового значения для Finger."""
    return (n + 2 ** i) % 2 ** m


def log_nodes(nodes) -> None:
    """Печать узлов в качестве логов."""
    print("Узлы:")
    for node in nodes:
        print(f"{str(node)}")


def stabilisation(nodes) -> None:
    """Стабилизация узлов в списке."""
    print("Стабилизируем узлы...")

    iterations = len(nodes) * len(nodes) + 1
    for i in tqdm(range(iterations)):
        for node in nodes:
            node.stabilize()
            node.fix_fingers()

    print("Стабилизация окончена")

    log_nodes(nodes)


In [4]:
from __future__ import annotations
from typing import Tuple

class Finger:
    """"Класс для хранения вспомогательной информации об узлах."""

    interval: Tuple[int, int]  # interval[0] - start, interval[1] - end
    node = None

    def __init__(self, n: int, m: int, i: int, node):
        """
        Инициализациия ифнормации об узлах.
        :param n: количество узлов
        :param m: количество бит, используемых для генерации идентификаторов
        :param i: индекс входа
        :param node: узел
        """
        self.__start = generate_finger_start(n, m, i)
        self.__end = generate_finger_start(n, m, i + 1)
        self.interval = (self.__start, self.__end)
        self.node = node

In [5]:
from __future__ import annotations
import random
from tabulate import tabulate
from typing import List, Optional

class ChordNode:
    """Реализация узла в алгоритме хорд."""

    id: int
    finger: List[Finger]

    def __init__(self, n: int, m: int):
        """
        Инициализация узла в алгоритме хорд.
        :param n: количество узлов в системе
        :param m: количество бит, используемых для генерации идентификаторов
        """
        self.id = n
        self.finger = [Finger(n, m, i, self) for i in range(0, m)]
        self.__predecessor = self

    def get_successor(self) -> ChordNode:
        """Выдает successor."""
        return self.finger[0].node

    def set_successor(self, node: ChordNode) -> None:
        """Устанавливает successor."""
        self.finger[0].node = node

    def get_predecessor(self) -> ChordNode:
        """Выдает predecessor."""
        return self.__predecessor

    def set_predecessor(self, node: Optional[ChordNode]) -> None:
        """Устанавливает predecessor."""
        self.__predecessor = node

    def find_successor(self, node_id: int) -> ChordNode:
        """Поиск successor по id."""
        node = self.find_predecessor(node_id)
        return node.get_successor()

    def find_predecessor(self, node_id: int) -> ChordNode:
        """Поиск predecessor по id."""
        node = self
        while not (self.__id_in_interval(node_id, node.id, node.get_successor().id)
                   or node_id == node.get_successor().id):
            node = node.closest_preceding_finger(node_id)
        return node

    def closest_preceding_finger(self, node_id: int) -> ChordNode:
        """Поиск ближайшего preceding finger."""
        m = len(self.finger)
        for i in range(m - 1, -1, -1):
            node: ChordNode = self.finger[i].node
            if self.__id_in_interval(node.id, self.id, node_id):
                return node
        return self

    def join(self, node: Optional[ChordNode]) -> None:
        """Добавление нового узла."""
        if node:
            self.init_finger_table(node)
            self.update_others()
        else:
            for i in range(len(self.finger)):
                self.finger[i].node = self
            self.set_predecessor(self)

    def init_finger_table(self, node: ChordNode) -> None:
        """Инициализация локальной таблицы finger."""
        self.finger[0].node = node.find_predecessor(self.finger[0].interval[0])
        successor = self.get_successor()
        self.set_predecessor(successor.get_predecessor())
        successor.set_predecessor(self)
        m = len(self.finger)
        for i in range(0, m):
            if self.__id_in_interval(self.finger[i + 1].interval[0], self.id, self.finger[i].node.id) \
                    or self.id == self.finger[i + 1].interval[0]:
                self.finger[i + 1].node = self.finger[i].node
            else:
                self.finger[i + 1].node = node.find_successor(self.finger[i + 1].interval[0])

    def update_others(self) -> None:
        """Обновление узлов, чьи таблицы finger относятся к узлу."""
        for i in range(0, len(self.finger)):
            index = (self.id - 2 ** i) % 2 ** len(self.finger)
            p = self.find_predecessor(node_id=index)
            p.update_finger_table(self, i)

    def update_finger_table(self, s: ChordNode, i: int) -> None:
        """Обновление таблицы информации об узлах."""
        if s.id == self.id or self.__id_in_interval(s.id, self.id, self.finger[i].node.id):
            self.finger[i].node = s
            p = self.get_predecessor()
            if p:
                p.update_finger_table(s, i)

    def join_to_node(self, node: Optional[ChordNode]) -> None:
        if node:
            self.set_predecessor(None)
            self.set_successor(node.find_successor(self.id))
        else:
            for finger in self.finger:
                finger.node = self
            self.set_predecessor(self)

    def stabilize(self) -> None:
        """Стабилизация системы."""
        x = self.get_successor().get_predecessor()
        if self.__id_in_interval(x.id, self.id, self.get_successor().id):
            self.set_successor(x)
        self.get_successor().notify(self)

    def notify(self, node: ChordNode) -> None:
        """Проверка на predecessor."""
        if self.get_predecessor() is None \
                or self.__id_in_interval(node.id, self.get_predecessor().id, self.id):
            self.set_predecessor(node)

    def fix_fingers(self) -> None:
        """Периодическое обновление таблицы finger."""
        i = random.randrange(len(self.finger))
        self.finger[i].node = self.find_successor(self.finger[i].interval[0])

    def __id_in_interval(self, node_id: int, start: int, end: int) -> bool:
        """Проверка, находится ли id в интервале [start, end)."""
        m = len(self.finger)
        _node_id = node_id
        _start = start
        _end = end
        if _start >= _end:
            _end += 2 ** m
            if _start > node_id:
                _node_id += 2 ** m
        return _start < _node_id < _end

    def delete(self) -> None:
        if self.get_predecessor():
            self.get_predecessor().set_successor(self.get_successor())
        self.get_successor().set_predecessor(self.get_predecessor())

        for i in range(len(self.finger)):
            j = self.id - 2 ** i
            p = self.find_predecessor(j)
            p.update_finger_table(self.get_successor(), i)

    def __str__(self):
        finger_table = [["interval", "node"], *[[finger.interval, finger.node.id] for finger in self.finger]]

        return f"\n{'=' * 20}\nid: {self.id}\nfinger_table:\n{tabulate(finger_table)}\n{'=' * 20}"

In [6]:

print("Базовый пример:")
m = 3

node_list = []
head_node = ChordNode(0, m)
head_node.join(None)
node_list.append(head_node)

for i in [1, 3]:
  chord_node = ChordNode(i, m)
  chord_node.join_to_node(head_node)
  node_list.append(chord_node)

print("Узлы созданы")
stabilisation(node_list)

print("Добавим узел...")
new_node = ChordNode(6, m)
new_node.join_to_node(head_node)
node_list.append(new_node)
stabilisation(node_list)
print("Узел добавлен")

print("Удаляем добавленный узел...")
node_list.remove(new_node)
new_node.delete()
stabilisation(node_list)
print("Узел удален успешно")

Базовый пример:
Узлы созданы
Стабилизируем узлы...


100%|██████████| 10/10 [00:00<00:00, 17810.21it/s]


Стабилизация окончена
Узлы:

id: 0
finger_table:
--------  ----
interval  node
(1, 2)    1
(2, 4)    3
(4, 0)    0
--------  ----

id: 1
finger_table:
--------  ----
interval  node
(2, 3)    3
(3, 5)    3
(5, 1)    0
--------  ----

id: 3
finger_table:
--------  ----
interval  node
(4, 5)    0
(5, 7)    0
(7, 3)    0
--------  ----
Добавим узел...
Стабилизируем узлы...


100%|██████████| 17/17 [00:00<00:00, 19229.55it/s]


Стабилизация окончена
Узлы:

id: 0
finger_table:
--------  ----
interval  node
(1, 2)    1
(2, 4)    3
(4, 0)    6
--------  ----

id: 1
finger_table:
--------  ----
interval  node
(2, 3)    3
(3, 5)    3
(5, 1)    6
--------  ----

id: 3
finger_table:
--------  ----
interval  node
(4, 5)    6
(5, 7)    6
(7, 3)    0
--------  ----

id: 6
finger_table:
--------  ----
interval  node
(7, 0)    0
(0, 2)    0
(2, 6)    3
--------  ----
Узел добавлен
Удаляем добавленный узел...
Стабилизируем узлы...


100%|██████████| 10/10 [00:00<00:00, 21108.73it/s]

Стабилизация окончена
Узлы:

id: 0
finger_table:
--------  ----
interval  node
(1, 2)    1
(2, 4)    3
(4, 0)    0
--------  ----

id: 1
finger_table:
--------  ----
interval  node
(2, 3)    3
(3, 5)    3
(5, 1)    0
--------  ----

id: 3
finger_table:
--------  ----
interval  node
(4, 5)    0
(5, 7)    0
(7, 3)    0
--------  ----
Узел удален успешно


In [8]:
NUMBER_NODE = 16
BYTE_LENGTH = 5
print(f"Создаем узлы в количестве: {NUMBER_NODE} и количество бит: {BYTE_LENGTH}")

node_list = []
head_node = ChordNode(0, BYTE_LENGTH)
head_node.join(None)
node_list.append(head_node)

for n in range(1, NUMBER_NODE):
  chord_node = ChordNode(n, BYTE_LENGTH)
  chord_node.join_to_node(head_node)
  node_list.append(chord_node)

print("Узлы созданы")
stabilisation(node_list)

print("Добавим узел...")
new_node = ChordNode(NUMBER_NODE + 1, BYTE_LENGTH)
new_node.join_to_node(head_node)
node_list.append(new_node)
stabilisation(node_list)
print("Узел добавлен")

print("Удаляем добавленный узел...")
node_list.remove(new_node)
new_node.delete()
stabilisation(node_list)
print("Узел удален успешно")

Создаем узлы в количестве: 16 и количество бит: 5
Узлы созданы
Стабилизируем узлы...


100%|██████████| 257/257 [00:00<00:00, 6508.45it/s]


Стабилизация окончена
Узлы:

id: 0
finger_table:
--------  ----
interval  node
(1, 2)    1
(2, 4)    2
(4, 8)    4
(8, 16)   8
(16, 0)   0
--------  ----

id: 1
finger_table:
--------  ----
interval  node
(2, 3)    2
(3, 5)    3
(5, 9)    5
(9, 17)   9
(17, 1)   0
--------  ----

id: 2
finger_table:
--------  ----
interval  node
(3, 4)    3
(4, 6)    4
(6, 10)   6
(10, 18)  10
(18, 2)   0
--------  ----

id: 3
finger_table:
--------  ----
interval  node
(4, 5)    4
(5, 7)    5
(7, 11)   7
(11, 19)  11
(19, 3)   0
--------  ----

id: 4
finger_table:
--------  ----
interval  node
(5, 6)    5
(6, 8)    6
(8, 12)   8
(12, 20)  12
(20, 4)   0
--------  ----

id: 5
finger_table:
--------  ----
interval  node
(6, 7)    6
(7, 9)    7
(9, 13)   9
(13, 21)  13
(21, 5)   0
--------  ----

id: 6
finger_table:
--------  ----
interval  node
(7, 8)    7
(8, 10)   8
(10, 14)  10
(14, 22)  14
(22, 6)   0
--------  ----

id: 7
finger_table:
--------  ----
interval  node
(8, 9)    8
(9, 11)   9
(11, 15) 

100%|██████████| 290/290 [00:00<00:00, 6549.37it/s]


Стабилизация окончена
Узлы:

id: 0
finger_table:
--------  ----
interval  node
(1, 2)    1
(2, 4)    2
(4, 8)    4
(8, 16)   8
(16, 0)   17
--------  ----

id: 1
finger_table:
--------  ----
interval  node
(2, 3)    2
(3, 5)    3
(5, 9)    5
(9, 17)   9
(17, 1)   17
--------  ----

id: 2
finger_table:
--------  ----
interval  node
(3, 4)    3
(4, 6)    4
(6, 10)   6
(10, 18)  10
(18, 2)   0
--------  ----

id: 3
finger_table:
--------  ----
interval  node
(4, 5)    4
(5, 7)    5
(7, 11)   7
(11, 19)  11
(19, 3)   0
--------  ----

id: 4
finger_table:
--------  ----
interval  node
(5, 6)    5
(6, 8)    6
(8, 12)   8
(12, 20)  12
(20, 4)   0
--------  ----

id: 5
finger_table:
--------  ----
interval  node
(6, 7)    6
(7, 9)    7
(9, 13)   9
(13, 21)  13
(21, 5)   0
--------  ----

id: 6
finger_table:
--------  ----
interval  node
(7, 8)    7
(8, 10)   8
(10, 14)  10
(14, 22)  14
(22, 6)   0
--------  ----

id: 7
finger_table:
--------  ----
interval  node
(8, 9)    8
(9, 11)   9
(11, 15

100%|██████████| 257/257 [00:00<00:00, 7409.46it/s]

Стабилизация окончена
Узлы:

id: 0
finger_table:
--------  ----
interval  node
(1, 2)    1
(2, 4)    2
(4, 8)    4
(8, 16)   8
(16, 0)   0
--------  ----

id: 1
finger_table:
--------  ----
interval  node
(2, 3)    2
(3, 5)    3
(5, 9)    5
(9, 17)   9
(17, 1)   0
--------  ----

id: 2
finger_table:
--------  ----
interval  node
(3, 4)    3
(4, 6)    4
(6, 10)   6
(10, 18)  10
(18, 2)   0
--------  ----

id: 3
finger_table:
--------  ----
interval  node
(4, 5)    4
(5, 7)    5
(7, 11)   7
(11, 19)  11
(19, 3)   0
--------  ----

id: 4
finger_table:
--------  ----
interval  node
(5, 6)    5
(6, 8)    6
(8, 12)   8
(12, 20)  12
(20, 4)   0
--------  ----

id: 5
finger_table:
--------  ----
interval  node
(6, 7)    6
(7, 9)    7
(9, 13)   9
(13, 21)  13
(21, 5)   0
--------  ----

id: 6
finger_table:
--------  ----
interval  node
(7, 8)    7
(8, 10)   8
(10, 14)  10
(14, 22)  14
(22, 6)   0
--------  ----

id: 7
finger_table:
--------  ----
interval  node
(8, 9)    8
(9, 11)   9
(11, 15) 